## Trying manual tokenization

In [11]:
# pip install keras, keras-bert

In [12]:
# pip install tensorflow

In [13]:
# coding: utf-8
import sys
import codecs
import numpy as np
from keras_bert import load_trained_model_from_checkpoint
import bert.tokenization as tokenization
import os

In [14]:
config_json = {
    "bert": 'C:\\Users\dannu\\bert-base',
}
config_json['bert_config'] = os.path.join(config_json['bert'], 'bert_config.json')
config_json['checkpoint'] = os.path.join(config_json['bert'], 'bert_model.ckpt')
config_json['vocab'] = os.path.join(config_json['bert'], 'vocab.txt')

: 

In [ ]:
tokenizer = tokenization.FullTokenizer(vocab_file=config_json['vocab'], do_lower_case=False)

In [ ]:
tokens = tokenizer.tokenize("""
The sun is nearly gone
The lights are turning on
A silver shine that stretches to the sea
We've stumbled on a view
That's tailor-made for two
What a shame those two are you and me
Some other girl and guy
Would love this swirling sky
But there's only you and I
And we've got no shot
This could never be
You're not the type for me (really?)
And there's not a spark in sight
What a waste of a lovely night
You say there's nothing here?
Well, let's make something clear
I think I'll be the one to make that call (but you'll call?)
And though you looked so cute
In your polyester suit (it's wool)
You're right, I'd never fall for you at all
And maybe this appeals
To someone not in heels
Or to any girl who feels
There's some chance for romance
But, I'm frankly feeling nothing
Is that so?
Or it could be less than nothing
Good to know, so you agree?
That's right
What a waste of a lovely night
""")

In [ ]:
token_input = tokenizer.convert_tokens_to_ids(tokens)   

In [ ]:
token_input

In [ ]:
tokenizer.convert_tokens_to_ids(["king"]) 

In [ ]:
tokenizer.convert_tokens_to_ids(["queen"])[0] - tokenizer.convert_tokens_to_ids("woman") [0]

## Actually, i'd rather try old thing, but a better version of it

In [4]:
# pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer("all-mpnet-base-v2")

### Extracting downloaded data

In [2]:
import json
DB_DIR = "db_lyrics"
remake = True
if remake:
    data = []
    for file in os.listdir(DB_DIR):
        with open(os.path.join(DB_DIR, file), 'r') as f:
            cur = json.load(f)
            for el in cur['data']:
                data.append((el['lyrics'], el['meta']))

In [58]:
import json
DB_DIR = "db_lyrics"
remake = True
if remake:
    song_data = []
    for file in os.listdir(DB_DIR):
        with open(os.path.join(DB_DIR, file), 'r') as f:
            cur = json.load(f)
            for el in cur['data']:
                song_data.append((el['song'], el['artist']))

In [60]:
len(song_data)

11559

### Encoding downloaded data

In [13]:
import tqdm
from threading import Thread

remake = False

if remake:
    def fill_encoded(index):
        encoded_data[index] = (transformer.encode(data[index][0]), data[index][1])

    step = 2
    for i in tqdm.trange(0, len(data), step):
        threads = [Thread(target=fill_encoded, args=(i + j,)) for j in range(min(step, len(data) - i))]
        for t in threads:
            t.start()
        for t in threads:
            t.join()
        # break
        # encoded_data[i] = (transformer.encode(data[i][0]), data[i][1])

100%|██████████| 5674/5674 [2:10:49<00:00,  1.38s/it]  


In [25]:
encoded_data[0][1].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'valence', 'tempo'])

### Saving encoded data

In [3]:
DATA_DIR = "all-mpnet-base-v2-data-new"
remake = False
keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'valence', 'tempo']
if remake:
    cur = []
    index = 0
    for el in encoded_data:
        cur.append({"lyrics": [x.item() for x in el[0]], "meta": [el[1][k] for k in keys]})
        # print(cur)
        # break
        if len(cur) >= 1000:
            filename = os.path.join(DATA_DIR, f"data-{index}.json")
            index += 1
            with open(filename, 'w') as f:
                json.dump(cur, f)
            cur = []
    filename = os.path.join(DATA_DIR, f"data-{index}.json")
    index += 1
    with open(filename, 'w') as f:
        json.dump(cur, f)
    cur = []

In [3]:
import json
import numpy as np
import torch
DATA_DIR = "all-mpnet-base-v2-data"
lyrics_max_val = 3.0483362674713135
meta_max_val = [torch.tensor(0.9880), torch.tensor(1.), torch.tensor(11.), torch.tensor(60.), torch.tensor(1.), torch.tensor(0.9970), torch.tensor(220.1690)]
    
vecs = []
for file in os.listdir(DATA_DIR):
    if file.endswith('.json'):
        filename = os.path.join(DATA_DIR, file)
        with open(filename, 'r') as f:
            cur = json.load(f)
            for el in cur:
                meta = el['meta']
                for j in range(len(meta)):
                    meta[j] /= meta_max_val[j]
                lyrics = el['lyrics']
                for j in range(len(lyrics)):
                    lyrics[j] /= lyrics_max_val
                vecs.append((np.array(lyrics, dtype=np.float32), np.array(meta, dtype=np.float32)))

### Rn scaling uses hardcoded values because the calculations of maximum values took to much time; it is the version that can be used to recalc values

In [ ]:
def scale_data():
    global dataset
    n = len(dataset)
    # may be scaling is not needed.
    first_time = False
    if first_time:
        lyrics_max_val = 0
        meta_max_val = [0 for i in range(7)]

        for data in tqdm(dataset):
        #     print(data[0])
        #     print(data[1])
    #         break
            lyrics_max_val = max(lyrics_max_val, max(data[0]).item())
            for i in range(7):
                meta_max_val[i] = max(meta_max_val[i], abs(data[1][i]))
    lyrics_max_val = 3.0483362674713135
    meta_max_val = [torch.tensor(0.9880), torch.tensor(1.), torch.tensor(11.), torch.tensor(60.), torch.tensor(1.), torch.tensor(0.9970), torch.tensor(220.1690)]
    print(lyrics_max_val)
    print(meta_max_val)


    first_time = True
    if first_time:
        # c = input("continue?(y/n)>>")
        c= 'y'
        if c.lower() == 'y':
            for data in tqdm(dataset):
                if lyrics_max_val > 1:
                    data[0] /= lyrics_max_val
                for i in range(7):
                    if meta_max_val[i] > 1:
                        data[1][i] /= meta_max_val[i]
    print(dataset[1])

### Translating data into np arrays

In [90]:
import numpy as np
vecs = [(np.array([x.item() for x in el[0]], dtype=np.float32), np.array([el[1][k] for k in keys], dtype=np.float32)) for el in encoded_data]

In [95]:
vecs[0][1].shape

(7,)

### Putting data into dataloaders

In [5]:
import torch

n = len(vecs)
train_set, test_set = [], []
train_loader, test_loader = [], []
def split_data():
    global train_set, test_loader, test_set, train_loader
    train_size = int(n * .9)
    train_set = vecs[:train_size]
    test_set = vecs[train_size:]
    # len(train_set), len(test_set)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=config_json['batch_size'], shuffle=True, pin_memory=True, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=config_json['batch_size'], shuffle=False)

# if __name__ == "__main__":
split_data()

## Training net

In [1]:
from model import Net
net = Net()

In [18]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import src.config as config
import time
def train_loop(model, criterion, optimizer, train_loader, n):
    loss_avg = AverageMeter()
    acc_stat = AverageMeter()
    start_time = time.time()
    for embeddings, targets in train_loader:
        model.train()
        model.zero_grad()
        embeddings = embeddings.to(DEVICE)
        targets = targets.to(DEVICE)
        output = model(embeddings)
        loss = criterion(output, targets)
        loss_avg.update(loss.item(), 1)

        output2 = output.softmax(dim=1)
        output2 = output2.cpu().detach().numpy()

        acc = np.linalg.norm(targets - output2)

        acc_stat.update(acc, 1)
        loss.backward()
        optimizer.step()
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    print(f"{n} epoch, Loss: {loss_avg.avg:.5f}, acc: {acc_stat.avg:.4f}, LR: {lr:.7f}, Time for cycle: {(time.time() - start_time):.2f} sec")
    return loss_avg.avg

import tqdm

def model_load(preTrained=False, checkpoint_path=""):
    model = Net()
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    if preTrained:
        """Loading model"""
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE )
        model.load_state_dict(     checkpoint['model'])
        model=model.to(DEVICE)
        # optimizer.load_state_dict( checkpoint['optimizer']) # i want to change lr
        epoch =                    checkpoint['epoch']
        score =                    checkpoint['score']
        return model, optimizer, criterion, epoch, score
    else:
        return model.to(DEVICE), optimizer, criterion, 0, 0


def save_model(model, optimizer, epoch, score, path):
    checkpoint={
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
        'score': score
    }
    torch.save(checkpoint, path)


def Train_log(score, path):
    arr=np.array([score])
    if os.path.exists(path):
        with open(path, 'rb') as f:
            arr = np.load(f)
        arr = np.append(arr, score)
    with open(path, 'wb') as f:
        np.save(f, arr)


def train(second_time):
    model, optimizer, criterion, epoch, score = model_load(config, second_time, config.CHECKPOINT_PATH)
    print("Model loaded")
    criterion = criterion.to(DEVICE)
    model=model.to(DEVICE)
    for e in range(100):
        score = train_loop(model, criterion, optimizer, train_loader, e)
        save_model(model, optimizer, e, score, config.CHECKPOINT_PATH)
        Train_log(score, config.TRAIN_LOG)
        print("Data saved \n")

In [56]:
train(False)

Model loaded
0 epoch, Loss: 0.02578, acc: 28.1108, LR: 0.0010000, Time for cycle: 0.62 sec
Data saved 

1 epoch, Loss: 0.01733, acc: 28.0538, LR: 0.0010000, Time for cycle: 0.55 sec
Data saved 

2 epoch, Loss: 0.01632, acc: 28.0184, LR: 0.0010000, Time for cycle: 0.55 sec
Data saved 

3 epoch, Loss: 0.01616, acc: 28.0095, LR: 0.0010000, Time for cycle: 0.58 sec
Data saved 

4 epoch, Loss: 0.01610, acc: 28.0133, LR: 0.0010000, Time for cycle: 0.54 sec
Data saved 

5 epoch, Loss: 0.01615, acc: 28.0088, LR: 0.0010000, Time for cycle: 0.53 sec
Data saved 

6 epoch, Loss: 0.01618, acc: 28.0156, LR: 0.0010000, Time for cycle: 0.56 sec
Data saved 

7 epoch, Loss: 0.01615, acc: 28.0091, LR: 0.0010000, Time for cycle: 0.58 sec
Data saved 

8 epoch, Loss: 0.01624, acc: 28.0164, LR: 0.0010000, Time for cycle: 0.54 sec
Data saved 

9 epoch, Loss: 0.01618, acc: 28.0142, LR: 0.0010000, Time for cycle: 0.55 sec
Data saved 

10 epoch, Loss: 0.01611, acc: 28.0243, LR: 0.0010000, Time for cycle: 0.54 se

### Saving model

In [7]:

if __name__ == "__main__":
    checkpoint = torch.load(config.CHECKPOINT_PATH, map_location=DEVICE)
    m = Net()
    m.load_state_dict(checkpoint['model'])
    torch.save(m.state_dict(), os.path.join(config.SAVE_DIR, f"{config.MODEL_NAME}.pt"))

### Just some testing

In [44]:
def test():
    def euclid_distance(a, b):
        assert len(a) == len(b)
        return np.sqrt(np.sum([
            (xa - xb) ** 2 for xa, xb, in zip(a, b)
        ]))


    _sum = 0
    num = 0
    mx = 0
    with torch.no_grad():
        
        for data in test_loader:
            X, y = data
            # print(y)
            output = m(X.view(-1,768))
            #print(output)
            for idx, res in enumerate(output):
                # print(y[idx])
                # print(res)
                _sum += euclid_distance(y[idx], res)
                num += 1


    print("Average distance: ", round(_sum/num, 3))
if __name__ == "__main__":
    test()


Average distance:  0.683


In [45]:
test_set[3][1]

array([ 0.6153846 ,  0.648     ,  0.09090909, -0.09281667,  1.        ,
        0.11434303,  0.5539381 ], dtype=float32)

In [65]:
data = test_set[:101]
import random
random.shuffle(data)
res = 0
for i in range(100):
    res = max(res, abs((m(torch.from_numpy(data[i + 1][0])) - m(torch.from_numpy(data[i][0])))[5]))
    # print(m(torch.from_numpy(data[i + 1][0])) - m(torch.from_numpy(data[i][0])))
print(res)

IndexError: index 7 is out of bounds for dimension 0 with size 7

### Merging lyrics and songs with vectors (?? I hope i won't need that in next version)

In [62]:
merged = []

for el, vec in zip(song_data, vecs):
    song, artist, meta = el[0], el[1], vec[1]
    merged.append({'song': song,
                   'artist': artist,
                   'meta': meta})

In [63]:
len(merged)

11559

In [69]:
for i in range(len(merged)):  # convert to list for serialization
    merged[i]['meta'] = list(map(float, merged[i]['meta']))

In [70]:
with open("meta+song.json", 'w') as f:
    json.dump(merged, f)